# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 2.3.4
- **Date :** 15/11/2023
- **Heure :** 23:43
- **Données :** `ready_Evaporation_Sunshine.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `to_sktime_dataset`
- **Algorithme :** ROCKET

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Omar/ready_Evaporation_Sunshine.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,2008,12,1,0,3.141593,3.141593,2.748894,-36.080477,146.916280,0
2008-12-01,5.8,17.6,3.2,52.0,26.0,30.0,71.0,37.0,1008.2,1007.5,...,2008,12,1,18,3.141593,3.141593,2.748894,-37.562303,143.860565,2
2008-12-02,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,2008,12,2,0,2.748894,1.963495,3.534292,-36.080477,146.916280,0
2008-12-02,4.3,19.5,0.4,44.0,20.0,24.0,66.0,47.0,1009.3,1007.3,...,2008,12,2,18,2.748894,2.748894,2.748894,-37.562303,143.860565,2
2008-12-03,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,2008,12,3,0,3.534292,3.141593,3.534292,-36.080477,146.916280,0


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10453 entries, 2008-12-01 to 2017-06-25
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         10453 non-null  float64
 1   MaxTemp         10453 non-null  float64
 2   Rainfall        10453 non-null  float64
 3   WindGustSpeed   10453 non-null  float64
 4   WindSpeed9am    10453 non-null  float64
 5   WindSpeed3pm    10453 non-null  float64
 6   Humidity9am     10453 non-null  float64
 7   Humidity3pm     10453 non-null  float64
 8   Pressure9am     10453 non-null  float64
 9   Pressure3pm     10453 non-null  float64
 10  Cloud9am        10453 non-null  float64
 11  Cloud3pm        10453 non-null  float64
 12  Temp9am         10453 non-null  float64
 13  Temp3pm         10453 non-null  float64
 14  RainToday       10453 non-null  int64  
 15  RainTomorrow    10453 non-null  int64  
 16  year            10453 non-null  int64  
 17  month           10453 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Conversion des jeux `X_train` et `X_test` au format `sktime` avec avec la fonction `to_sktime_dataset` de la bibliothèque `tslearn`

from tslearn.utils import to_sktime_dataset

X_train = to_sktime_dataset(X_train)
X_test = to_sktime_dataset(X_test)

## 4. Modélisation

In [8]:
# Instanciation d'un transformateur ROCKET

from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()

In [9]:
# Récupération des paramètres initiaux du transformateur ROCKET

rocket.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 10000, 'random_state': None}

In [10]:
# Ajustement du transformateur ROCKET au jeu d'entraînement

rocket.fit(X_train)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Rocket()

In [11]:
# Récupération des paramètres ajustés du transformateur ROCKET

rocket.get_fitted_params()

{}

In [12]:
# Application du transformateur ROCKET au jeu d'entraînement

X_train_transform = rocket.transform(X_train)

In [13]:
# Instanciation d'un modèle classificateur linéaire

from sklearn.linear_model import RidgeClassifierCV

clf_rocket_ridge = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10)) ## Pour ce paramétrage initial, nous suivons l'exemple fourni par `sktime`, qui fixe 10 alphas, de -3 à 3, à équidistance sur l'échelle logarithmique.

In [14]:
# Récupération des paramètres initiaux du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [15]:
# Entraînement du modèle

clf_rocket_ridge.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [16]:
# Récupération des paramètres ajustés du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [17]:
# Application du transformateur ROCKET au jeu de test

X_test_transform = rocket.transform(X_test)

In [18]:
# Réalisation des prédictions

y_pred = clf_rocket_ridge.predict(X_test_transform)

## 5. Évaluation

In [19]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1304,72
1,168,198


In [20]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1376
           1       0.73      0.54      0.62       366

    accuracy                           0.86      1742
   macro avg       0.81      0.74      0.77      1742
weighted avg       0.85      0.86      0.85      1742

